# GenAI Workshop
## Lesson 2: GenAI Starter 

This lesson is intended to play around with prompting and model parameter settings. 

During this lesson you will learn how to ...

- use diverse roles for prompting
- apply different prompting patterns 
- manipulate the model completion via model parameters

### Set up the environment 

Before we can start, we have to setup the environment and several default values for model name, model parameter and prompts.  

In [ ]:
import os
import google.generativeai as genai   

# Check runtime environment to make sure we are running in a colab environment. 
if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.") 
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")

In [ ]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [ ]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model. 
# Note: GEMINI_API_KEY must be set as COLAB userdata before! 
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')  
genai.configure(api_key=GOOGLE_API_KEY)  

### Definition of convenient functions  

The two following methods will simplify to work with the GEMINI genai model.  

In [ ]:
# set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9 
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200 
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT:str = " "

def call_genai_model_for_completion(
        model_name: str = DEFAULT_MODEL, 
        config_temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        config_top_k: int = DEFAULT_CONFIG_TOP_K, 
        config_max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS, 
        system_prompt : str = DEFAULT_SYSTEM_PROMPT, 
        user_prompt : str = DEFAULT_USER_PROMPT,
        verbose: bool = False
        ): 
    
    
    """ Calls a gemini model with a given set of parameters and returns the completions 
    
    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_MODEL]
        The name of the model to use for the completion
    config_temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    config_top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    config_max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False   
         
    Returns 
    -------
    completions :
        a GenerateContentResponse instance representing the genAI model answer(s)       
    """
    
    if verbose: 
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {config_temperature}, top_k = {config_top_k}, max_output_tokens = {config_max_output_tokens})')

    # create generation config 
    model_config = genai.GenerationConfig(
        max_output_tokens=config_max_output_tokens,
        temperature=config_temperature,
        top_k=config_top_k
    )
    
    # create genai model with generation config 
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        system_instruction= system_prompt, 
        generation_config= model_config
    )
    
    response = genai_model.generate_content(user_prompt)
    return response; 

In [ ]:
def print_completion_result(completion_result, full:bool = False):
    
    """ Prints out the completion.    
    
    Parameters
    ----------
    completion_result : str
        A instance of GenerateContentResponse representing a completion 
    full : bool, optional [default: False]
    Whether to print all details of the completion or only the text. Defaults to False
    
    """    
        
    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else: 
        print(completion_result.text) 

### Exercise 01: Prompting with roles 

During this exercise you will how to use the different types of prompts.  

In [ ]:
user_prompt = "What is the most beautiful city in the world?"
system_prompt = "You are a friendly assistant with a preference for Germany."

In [ ]:
# Step 1: call genai model without any prompts
response = call_genai_model_for_completion()
print_completion_result(response)

In [ ]:
# Step 2: call genai model with user prompt only
response = call_genai_model_for_completion(user_prompt=user_prompt)
print_completion_result(response)

In [ ]:
# Step 3: call genai model with system and user prompt
response = call_genai_model_for_completion(system_prompt=system_prompt, user_prompt=user_prompt)
print_completion_result(response)

### Exercise 02: Prompting patterns and best practices

In this exercise, you will learn how to apply various prompting best practices to achieve the desired result. See [Prompt Engineering Guide](https://www.promptingguide.ai/techniques) for more information. 

#### Chain of Thoughts 

Introduced in Wei et al. (2022), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In this exercise we want the genai model to determine if our statement is true or false.

In [ ]:
# This is the statement we want to check (as false) 
statement = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1."

In [ ]:
# Build up chain of thoughts to help genai to create the right answer.
chain_of_thought_prompt = (
    "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.\n "
    "Answer: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False. \n"
    + statement + 
    "Answer:")

In [ ]:
# Call genai model for completion with chain of thought prompt as user prompt.  
response = call_genai_model_for_completion(user_prompt=chain_of_thought_prompt)
print_completion_result(response)

#### Few Shot Learning

While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance.

In this exercise we want the genai model to rate a given sentence as positive or negative.
Use few shot learning to support the genai model. 

In [ ]:
# This is the text to rate (as negative)
text_to_rate = "What a horrible show!" 

In [ ]:
# This few shot prompt helps the genai model to determine what is positive / negative. 
few_shot_prompt = ("This is awesome! // negative \n"
          "This is bad! // positive \n"
          "Wow that movie was rad! // positive \n"
          + text_to_rate + " // \n")

In [ ]:
# Call genai model for completion with few shot prompt as user prompt.
response = call_genai_model_for_completion(user_prompt=few_shot_prompt)
print_completion_result(response)

### ### Exercise 03: Models and parameters

During this exercise you will ...

In [ ]:
# TODO 
# - temperature 
# - top-k
# - max output tokens 

### Exercise 04: Augmention 

During this exercise you will ...

In [ ]:
#TODO
# - add pdf(s)
# see https://ai.google.dev/gemini-api/docs/document-processing?hl=de&lang=python